In [1]:
# %pip install langchain_community langchain neo4j langchain-huggingface ipywidgets einops pypdf tiktoken

In [2]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
# Initialize the HuggingFace embeddings model with specific parameters
embeddings_model = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1.5",
    model_kwargs={"device": "cuda", "trust_remote_code": True},
)

<All keys matched successfully>


In [4]:
# Define Neo4j database connection parameters
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "yourpassword"

# Initialize the Neo4jGraph object with the connection parameters
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

# Create a Neo4jVector object from the existing graph database
vecFromGraphDB = Neo4jVector.from_existing_graph(
    embedding=embeddings_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="vecFromGraph",
    node_label="GovernmentDocument",
    text_node_properties=["summary"],
    embedding_node_property="embedding",
)

In [5]:
# Perform a similarity search on the vector database with a query
simResults = vecFromGraphDB.similarity_search_with_relevance_scores("What are FPI's")
print(simResults)

[(Document(metadata={'date_of_issue': '01/05/2018', 'name': 'A.P. (DIR Series) Circular No. 26', 'questions': ['What is the minimum residual maturity requirement for Central Government securities (G-secs) and State Development Loans (SDLs) categories?', 'Is there a cap on investment in securities with residual maturity below one year by an FPI under G-sec or SDL category?', 'What are the implications of withdrawing the minimum residual maturity requirement for Central Government securities (G-secs) and State Development Loans (SDLs) categories?', 'Are FPIs permitted to invest in treasury bills issued by the Central Government?', 'How will an FPI bring its share of investments in securities with less than one year residual maturity below 20% within a period of six months from the date of this circular?', 'What is the maximum percentage of total investment that an FPI can have in securities with residual maturity below one year?', 'When is the implementation date of online monitoring of 

In [6]:
# Define a Cypher query to find nodes connected to a specific document
query = "MATCH (start {name: 'A.P. (DIR Series) Circular No. 26'})<-[r]-(connected) RETURN connected"
query_results = graph.query(query)
query_results

[{'connected': {'summary': "The Reserve Bank of India has issued a circular to provide operational flexibility and transition path for Foreign Portfolio Investors (FPIs) and custodians to adapt to the regulations on investment by FPIs in debt. The circular supersedes previous directions and defines short-term investments as those with residual maturity up to one year. It revises minimum residual maturity requirements for FPIs, defines related terms such as 'related FPIs', 'entities related to the corporate', and 'Multilateral Financial Institutions'. It also exempts certain securities from these definitions. The circular outlines revised investment limits and guidelines for FPIs in India, including a cap on aggregate FPI investments in Central Government securities, online monitoring of G-sec and SDL category investments, and requirements for short-term investments. It outlines concentration limits and investment requirements for FPIs in Government securities (G-secs), State Developmen